In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"
cohort = "GSE248514"

# Input paths
in_trait_dir = "../../input/GEO/Mesothelioma"
in_cohort_dir = "../../input/GEO/Mesothelioma/GSE248514"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/GSE248514.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/GSE248514.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/GSE248514.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE248514_family.soft.gz', 'GSE248514_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mesothelioma/GSE248514/GSE248514_family.soft.gz
Matrix file: ../../input/GEO/Mesothelioma/GSE248514/GSE248514_series_matrix.txt.gz
Background Information:
!Series_title	"Coupling of response biomarkers between tumour and peripheral blood in mesothelioma patients undergoing chemoimmunotherapy"
!Series_summary	"Platinum-based chemotherapy in combination with anti-PD-L1 antibodies has shown promising results in mesothelioma. However, the immunological mechanisms underlying its efficacy are not well understood and there are no predictive biomarkers of clinical outcomes to guide treatment decisions. Here, we combine time-course RNA sequencing of peripheral blood mononuclear cells with pre-treatment tumour transcriptome data from the 54-patient cohort in the single arm phase II DREAM study. The identified immunological correlates are predictive of response and provide further evi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series_summary and Series_overall_design, this dataset contains gene expression data
# from nanoString nCounter platform for 770 immune-oncology related genes
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: Mesothelioma
# From the sample characteristics dictionary, we can use progression-free at 6 months as a trait
# since the study focuses on response biomarkers, and this is a clinically relevant outcome
trait_row = 5  # 'progression-free at 6 months'

# Age: Not explicitly provided in the sample characteristics
age_row = None  # Age data is not available

# Gender: Available in the sample characteristics
gender_row = 3  # 'gender'

# 2.2 Data Type Conversion

# For trait (progression-free at 6 months)
def convert_trait(value):
    if value is None:
        return None
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() == 'yes':
        return 1  # progression-free
    elif value.lower() == 'no':
        return 0  # not progression-free
    return None

# For gender
def convert_gender(value):
    if value is None:
        return None
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    return None

# Convert age function (not used, but defined for completeness)
def convert_age(value):
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Initial filtering validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Use the clinical data that should be available from previous steps
    # Typically accessed via a global variable or passed to this function
    
    # Extract clinical features from clinical_data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # Assuming clinical_data is already available from previous steps
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical data:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of selected clinical data:
{'GSM7916142': [0.0, 1.0], 'GSM7916143': [0.0, 1.0], 'GSM7916144': [0.0, 1.0], 'GSM7916145': [0.0, 1.0], 'GSM7916146': [0.0, 0.0], 'GSM7916147': [0.0, 1.0], 'GSM7916148': [1.0, 1.0], 'GSM7916149': [1.0, 1.0], 'GSM7916150': [0.0, 1.0], 'GSM7916151': [1.0, 0.0], 'GSM7916152': [0.0, 1.0], 'GSM7916153': [0.0, 1.0], 'GSM7916154': [1.0, 1.0], 'GSM7916155': [0.0, 1.0], 'GSM7916156': [1.0, 1.0], 'GSM7916157': [0.0, 1.0], 'GSM7916158': [1.0, 1.0], 'GSM7916159': [0.0, 1.0], 'GSM7916160': [1.0, 0.0], 'GSM7916161': [1.0, 1.0], 'GSM7916162': [0.0, 1.0], 'GSM7916163': [1.0, 1.0], 'GSM7916164': [1.0, 1.0], 'GSM7916165': [1.0, 1.0], 'GSM7916166': [1.0, 1.0], 'GSM7916167': [1.0, 0.0], 'GSM7916168': [0.0, 1.0], 'GSM7916169': [0.0, 1.0], 'GSM7916170': [1.0, 1.0], 'GSM7916171': [0.0, 1.0], 'GSM7916172': [0.0, 1.0], 'GSM7916173': [1.0, 1.0], 'GSM7916174': [0.0, 1.0], 'GSM7916175': [0.0, 1.0], 'GSM7916176': [1.0, 0.0], 'GSM7916177': [1.0, 1.0], 'GSM7916178': [1.0, 1.0], 'G

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.

Gene data extraction result:
Number of rows: 784
First 20 gene/probe identifiers:
Index(['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1',
       'ALDOA', 'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B',
       'API5', 'APLNR', 'APOE', 'APOL6'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Gene Identifier Review

# Observe the gene identifiers provided in the previous step
# The identifiers in the dataset are: 'A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', etc.

# Based on biomedical knowledge, these appear to be human gene symbols (official gene symbols)
# rather than other identifiers like Affymetrix probe IDs, Ensembl IDs, or RefSeq IDs.
# They follow the standard HGNC (HUGO Gene Nomenclature Committee) naming conventions.

# Therefore, no mapping to gene symbols is required

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Load the clinical data previously preprocessed (from Step 2)
# Use the clinical features previously extracted with the correct trait_row (5)
selected_clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data from {out_clinical_data_file}")
print(f"Shape of clinical data: {selected_clinical_df.shape}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# Check if any samples remain after missing value handling
if linked_data.shape[0] == 0:
    print("Warning: No samples remain after handling missing values. Check if trait column has valid data.")
    is_usable = False
else:
    # 5. Determine if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

    # 6. Validate the dataset and save cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note="Dataset contains gene expression data from mesothelioma patients undergoing chemoimmunotherapy."
    )

    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")

Top 10 gene indices before normalization: ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC']
Top 10 gene indices after normalization: ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC']
Shape of normalized gene data: (762, 46)
Saved normalized gene data to ../../output/preprocess/Mesothelioma/gene_data/GSE248514.csv
Loaded clinical data from ../../output/preprocess/Mesothelioma/clinical_data/GSE248514.csv
Shape of clinical data: (2, 46)
Shape of linked data: (46, 764)


Shape of linked data after handling missing values: (46, 764)
For the feature 'Mesothelioma', the least common label is '0.0' with 22 occurrences. This represents 47.83% of the dataset.
The distribution of the feature 'Mesothelioma' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 6 occurrences. This represents 13.04% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Mesothelioma/GSE248514.csv
